# 1. Loading & counting

In [1]:
import pandas as pd
import os

In [2]:
#get excel file
xlsx = pd.read_excel('~/wd_speciesid/data/raw/BtF-information/Beyond_the_Fence_ALL_TAGS_to_Feb22.xlsx',
                     header=1)
xlsx

,RecordID,Image Name,Site,Treatment,Date,Time,Moon Phase,Temp,Comments,Number,Pouch status,Species
0,BFGrid01874,2021-05-24 17-58-33 M 1_3.JPG,PCAM01,Park,2021-05-24,17:58:33,Waxing Gibbous,21,NaN,2.0,NaN,Red Kangaroo
1,BFGrid01880,2021-05-24 18-02-50 M 2_3.JPG,PCAM01,Park,2021-05-24,18:02:50,Waxing Gibbous,21,NaN,1.0,NaN,Red Kangaroo
2,BFGrid01881,2021-05-24 22-38-03 M 1_3.JPG,PCAM01,Park,2021-05-24,22:38:03,Waxing Gibbous,16,NaN,2.0,NaN,Red Kangaroo
3,BFGrid01890,2021-05-24 21-27-09 M 1_3.JPG,PCAM01,Park,2021-05-24,21:27:09,Waxing Gibbous,17,NaN,2.0,NaN,Red Kangaroo
4,BFGrid01891,2021-05-24 21-26-03 M 3_3.JPG,PCAM01,Park,2021-05-24,21:26:03,Waxing Gibbous,17,NaN,2.0,NaN,Red Kangaroo
...,...,...,...,...,...,...,...,...,...,...,...,...
15351,BFGrid14116,2021-12-27 01-44-43 M 1_3.JPG,WCAM15,WTZ,2021-12-27,01:44:43,Last Quarter,23,NaN,1.0,NaN,Red Kangaroo
15352,BFGrid14117,2021-12-27 03-05-32 M 1_3.JPG,WCAM15,WTZ,2021-12-27,03:05:32,Last Quarter,22,NaN,1.0,NaN,Red Kangaroo
15353,BFGrid14494,2022-01-14 23-20-32 M 1_3.JPG,WCAM15,WTZ,2022-01-14,23:20:32,Waxing Gibbous,23,NaN,1.0,NaN,Red Kangaroo
15354,BFGrid14534,2022-01-15 02-37-22 M 3_3.JPG,WCAM15,WTZ,2022-01-15,02:37:22,Waxing Gibbous,19,NaN,1.0,NaN,Red Kangaroo


In [4]:
xlsx['Species'].unique()

array(['Red Kangaroo', 'Kangaroo', 'Dingo', 'Rabbit', 'Cat', 'Emu',
       'Bird', 'Pig', 'Euro', 'Fox', 'Echidna', 'Western Grey Kangaroo',
       'Small mammal', 'Other', 'Goat'], dtype=object)

In [5]:
xlsx['Species'].value_counts()

Red Kangaroo             8776
Cat                      2378
Kangaroo                 1375
Rabbit                   1218
Bird                      539
Small mammal              457
Western Grey Kangaroo     165
Dingo                     131
Emu                       120
Euro                       56
Pig                        50
Fox                        35
Goat                       25
Echidna                    18
Other                      13
Name: Species, dtype: int64

# 2 Deciding how to split

In [18]:
#check the dates
for species in xlsx['Species'].unique():
    new_df = xlsx[xlsx['Species']==species]
    min_date = min(new_df['Date'])
    max_date = max(new_df['Date'])
    print(species, ': min date is', min_date, '& max date is', max_date)


Red Kangaroo : min date is 2021-02-10 00:00:00 & max date is 2022-02-17 00:00:00
Kangaroo : min date is 2021-02-11 00:00:00 & max date is 2022-02-12 00:00:00
Dingo : min date is 2021-04-03 00:00:00 & max date is 2022-02-17 00:00:00
Rabbit : min date is 2021-02-07 00:00:00 & max date is 2022-02-15 00:00:00
Cat : min date is 2021-02-11 00:00:00 & max date is 2022-02-18 00:00:00
Emu : min date is 2021-02-14 00:00:00 & max date is 2022-02-15 00:00:00
Bird : min date is 2021-02-15 00:00:00 & max date is 2021-10-04 00:00:00
Pig : min date is 2021-03-31 00:00:00 & max date is 2021-12-14 00:00:00
Euro : min date is 2021-06-09 00:00:00 & max date is 2022-01-24 00:00:00
Fox : min date is 2021-03-20 00:00:00 & max date is 2021-07-26 00:00:00
Echidna : min date is 2021-03-14 00:00:00 & max date is 2021-12-19 00:00:00
Western Grey Kangaroo : min date is 2021-04-23 00:00:00 & max date is 2021-12-16 00:00:00
Small mammal : min date is 2021-02-14 00:00:00 & max date is 2022-02-13 00:00:00
Other : min 

In [158]:
train_entries = xlsx[xlsx['Date']<'2021-10-01 00:00:00'] #Feb2021 to Sep2021
print(len(train_entries))
val_entries = xlsx[ (xlsx['Date']>'2021-09-30 00:00:00') & (xlsx['Date']<'2021-12-01 00:00:00') ] #Oct2021 to Nov2021
print(len(val_entries))
test_entries = xlsx[xlsx['Date']>'2021-11-30 00:00:00'] #from Dec2021
print(len(test_entries))

10029
3089
2238


In [184]:
all_species = xlsx['Species'].unique()

for species in all_species:
    train = len(train_entries[train_entries['Species']==species])
    val = len(val_entries[val_entries['Species']==species])
    test = len(test_entries[test_entries['Species']==species])
    print(species, ':', train, '/', val, '/', test)

Red Kangaroo : 5159 / 2268 / 1349
Kangaroo : 1245 / 67 / 63
Dingo : 75 / 8 / 48
Rabbit : 790 / 198 / 230
Cat : 1425 / 455 / 498
Emu : 69 / 27 / 24
Bird : 538 / 1 / 0
Pig : 47 / 2 / 1
Euro : 26 / 25 / 5
Fox : 35 / 0 / 0
Echidna : 11 / 5 / 2
Western Grey Kangaroo : 140 / 22 / 3
Small mammal : 438 / 11 / 8
Other : 13 / 0 / 0
Goat : 18 / 0 / 7


In [3]:
import pandas as pd
import numpy as np

# Group data by 'Species'
grouped = xlsx.groupby('Species')

# Initialize dictionaries to store train, validation, and test data
train_data = []
val_data = []
test_data = []

# Loop through each species group
for species, group in grouped:
    # Sort images by date
    group = group.sort_values(by='Date')

    # Calculate split sizes
    total_size = len(group)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size

    # Split images
    train_data.extend(group[:train_size])
    val_data.extend(group[train_size:train_size + val_size])
    test_data.extend(group[train_size + val_size:])

# Create new DataFrames for train, validation, and test sets
train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)
test_df = pd.DataFrame(test_data)

# Now you have separate DataFrames for each split


In [55]:
import pandas as pd
import numpy as np

# Initialize dictionaries to store train, validation, and test data
train_data = []
val_data = []
test_data = []

# Loop through each species group
for species in xlsx['Species'].unique():
    # Sort images by date
    new_df = xlsx[xlsx['Species']==species]
    group = new_df.sort_values('Date')

    # Calculate split sizes
    total_size = len(group)
    train_size = int(0.7 * total_size)
    val_size = int(0.15 * total_size)
    test_size = total_size - train_size - val_size

    # Split images
    train_data.append(group[:train_size])
    val_data.append(group[train_size:train_size + val_size])
    test_data.append(group[train_size + val_size:])

# Create new DataFrames for train, validation, and test sets
# train_df = pd.DataFrame(train_data)
# val_df = pd.DataFrame(val_data)
# test_df = pd.DataFrame(test_data)

# Now you have separate DataFrames for each split


In [80]:
train_data.

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (15,) + inhomogeneous part.

## 2.2. Split with scikitlearn

In [ ]:
from sklearn.model_selection import train_test_split

#train / test
X_train, X_test, y_train, y_test 
    = train_test_split(X, y, test_size=0.2, random_state=1)

#train / val
 X_train, X_val, y_train, y_val 
    = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

# ?. Moving the images around

In [22]:
#create parent directory for processed images
image_path = '/media/jess/DATA2/PhD/data/wild_deserts/'
new_folder = 'processed/'
processed_path = os.path.join(image_path, new_folder)
try:    
    os.mkdir(processed_path)
    print('Done!')
except:
    print('Folder created already!')

Done!


In [31]:
#create a folder for each species
def create_species_folder(species):
    species_folder = species + '/'
    species_path = os.path.join(processed_path, species_folder)
    try:    
        os.mkdir(species_path)
        print(species + '- Done!')
    except:
        print('Folder created already!')

In [32]:
species = xlsx['Species'].unique()
for sp in species:
    create_species_folder(sp)

Red Kangaroo- Done!
Kangaroo- Done!
Dingo- Done!
Rabbit- Done!
Cat- Done!
Emu- Done!
Bird- Done!
Pig- Done!
Euro- Done!
Fox- Done!
Echidna- Done!
Western Grey Kangaroo- Done!
Small mammal- Done!
Other- Done!
Goat- Done!
